In [0]:
"""   Neste notebook realizei a transformação dos dados do CPGF a partir da camada Bronze para a camada Silver, com foco em estruturação, padronização e qualidade dos dados.
"""

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

df_bronze = spark.table("default.bronze_cpgf")


In [0]:
item_schema = StructType([
    StructField("id", LongType()),
    StructField("codOrgaoSuperior", LongType()),
    StructField("nomOrgaoSuperior", StringType()),
    StructField("codOrgao", LongType()),
    StructField("nomOrgao", StringType()),
    StructField("codUnidadeGestora", LongType()),
    StructField("nomUnidadeGestora", StringType()),
    StructField("numAnoExtrato", IntegerType()),
    StructField("numMesExtrato", IntegerType()),
    StructField("numCpfPortador", StringType()),
    StructField("nomPortador", StringType()),
    StructField("numCnpjCpfFavorecido", StringType()),
    StructField("nomFavorecido", StringType()),
    StructField("numTransacao", StringType()),
    StructField("datTransacao", StringType()),
    StructField("vlrTransacao", StringType())
])


In [0]:
df_parsed = df_bronze.withColumn(
    "json_array",
    F.from_json(F.col("json"), ArrayType(item_schema))
)


In [0]:
df_exploded = df_parsed.withColumn(
    "item",
    F.explode("json_array")
)


In [0]:
df_silver = df_exploded.select(
    F.col("item.id").alias("id_transacao"),
    F.col("item.codOrgaoSuperior"),
    F.col("item.nomOrgaoSuperior"),
    F.col("item.codOrgao"),
    F.col("item.nomOrgao"),
    F.col("item.codUnidadeGestora"),
    F.col("item.nomUnidadeGestora"),
    F.col("item.numAnoExtrato").alias("ano"),
    F.col("item.numMesExtrato").alias("mes"),
    F.col("item.numCpfPortador").alias("cpf_portador"),
    F.col("item.nomPortador").alias("nome_portador"),
    F.col("item.numCnpjCpfFavorecido").alias("cpf_cnpj_favorecido"),
    F.col("item.nomFavorecido").alias("nome_favorecido"),
    F.col("item.numTransacao").alias("tipo_transacao"),
    F.col("item.datTransacao").alias("data_transacao"),
    F.col("item.vlrTransacao").alias("valor_transacao_raw"),
    F.current_timestamp().alias("ingest_ts")
)


In [0]:
df_silver = (
    df_silver
    .withColumn(
        "valor_transacao",
        F.regexp_replace("valor_transacao_raw", ",", ".").cast("decimal(15,2)")
    )
    .withColumn(
        "data_transacao",
        F.to_date("data_transacao")
    )
)


In [0]:
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_cpgf")


In [0]:
spark.table("silver_cpgf").limit(5).display()

id_transacao,codOrgaoSuperior,nomOrgaoSuperior,codOrgao,nomOrgao,codUnidadeGestora,nomUnidadeGestora,ano,mes,cpf_portador,nome_portador,cpf_cnpj_favorecido,nome_favorecido,tipo_transacao,data_transacao,valor_transacao_raw,ingest_ts,valor_transacao
1,20000,Presidência da República,20415,Empresa Brasil de Comunicação,115406,EMPRESA BRASIL DE COMUNICACAO S.A,2025,8,***.801.911-**,DANILO FERREIRA DE SOUSA,-1,SEM INFORMACAO,COMPRA A/V - INT$ - APRES,2025-06-30,"32,23",2025-12-14T03:19:15.301Z,32.23
2,20000,Presidência da República,20415,Empresa Brasil de Comunicação,115406,EMPRESA BRASIL DE COMUNICACAO S.A,2025,8,***.801.911-**,DANILO FERREIRA DE SOUSA,-1,SEM INFORMACAO,COMPRA A/V - INT$ - APRES,2025-06-30,"25,97",2025-12-14T03:19:15.301Z,25.97
3,20000,Presidência da República,20415,Empresa Brasil de Comunicação,115406,EMPRESA BRASIL DE COMUNICACAO S.A,2025,8,***.759.938-**,ALEX SANDRO DE JESUS SILVA,-1,SEM INFORMACAO,COMPRA A/V - INT$ - APRES,2025-07-02,"25,74",2025-12-14T03:19:15.301Z,25.74
4,20000,Presidência da República,20415,Empresa Brasil de Comunicação,115406,EMPRESA BRASIL DE COMUNICACAO S.A,2025,8,***.759.938-**,ALEX SANDRO DE JESUS SILVA,-1,SEM INFORMACAO,COMPRA A/V - INT$ - APRES,2025-07-02,"32,65",2025-12-14T03:19:15.301Z,32.65
5,20000,Presidência da República,20415,Empresa Brasil de Comunicação,115406,EMPRESA BRASIL DE COMUNICACAO S.A,2025,8,***.759.938-**,ALEX SANDRO DE JESUS SILVA,-1,SEM INFORMACAO,COMPRA A/V - INT$ - APRES,2025-06-30,"26,94",2025-12-14T03:19:15.301Z,26.94
